<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/AutoCaptioner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BLIP-2 CAPTIONING SERVICE

Auto-captioning service using latest Blip-2. Requires high ram.  

[![Buy me a beer](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/buy-me-a-beer.png ) ](https://steinhaug.com/donate/)

__NB!__  
This Notebook is not made for "people in a hurry", so you need to make sure to download your imnages and make sure the inference loop is targeting your images.  
That being said, the notebook was used by me for creating captions but ask me and I'll put together a 1,2,3 GO notebook.  

In [1]:
from IPython.display import clear_output
#@title Connect runtime and mount Google Drive
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

from google.colab import drive
drive.mount('/content/drive')

Tesla V100-SXM2-16GB, 16384 MiB, 16150 MiB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
IMAGE_FOLDERS = '/content/drive/MyDrive/datasets/metart'

## Captioning pipeline

In [2]:
#@markdown Install blip-2
from IPython.display import clear_output
import sys
if 'google.colab' in sys.modules:
    !pip3 install salesforce-lavis
clear_output()
print('[1;32mDone! ✓')

Done! ✓


In [3]:
#@markdown Init blip-2
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
#@title Load the blip-2 model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt2.7b", is_eval=True, device=device
)
#model, vis_processors, _ = load_model_and_preprocess(
#    name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
#)
vis_processors.keys()
clear_output(); print('[1;32mDone! ✓')

In [48]:
#@markdown Functions
import os; import random; import json; import time;
from os.path import normpath, basename; import zipfile;

def create_zip_from_directory(zip_filename, directory_path, use_directory_inside_zip=True):
    if use_directory_inside_zip:
        curr_directory = basename(normpath(directory_path))
    with zipfile.ZipFile(zip_filename, 'w') as zip_file:
        for root, _, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                archive_path = os.path.relpath(file_path, directory_path)
                if use_directory_inside_zip:
                    zip_file.write(file_path, os.path.join(curr_directory, archive_path))
                else:
                    zip_file.write(file_path, archive_path)
    return ret_zipFileCount(zip_filename);

def ret_zipFileCount(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as zip_file:
        file_count = len(zip_file.namelist())
        return file_count
        #print(f"Number of files in '{zip_filename}': {file_count}")

def ret_directoryFileCount(dir_path):
    file_count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            file_count += 1
    return file_count

def load_image(image_path, sizeDiv=8):
    raw_image = Image.open(f'{image_path}').convert('RGB')
    h, w = raw_image.size
    display(raw_image.resize(( int(h / sizeDiv) , int(w / sizeDiv) )))
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    return image

def wrap_query(query):
    return "Question: " + query + " Answer:"

def write__logfile(file, msg):
    with open(file, 'a') as fa:
        fa.write(msg)
    return ''

def write_the_file(path, data_string):
    if len(str(data_string)):
        with open(path, 'w+') as fw:
            fw.write(str(data_string))
    else:
        if os.path.exists(path):
            os.remove(path)
    return '';

def ret__longest_val(arr):
    curr_v = ''
    curr_x = 0
    while arr:
        val = arr.pop(0)
        if len(val) >= curr_x:
            curr_x = len(val)
            curr_v = val
    return curr_v

def auto__get_AI_caption(IMG_SRC):
    image = load_image(IMG_SRC)
    cap1 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=False, num_captions=4) )
    cap2 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=True, num_captions=4) )
    cap3 = model.generate({"image": image, "prompt": wrap_query('What is out of the ordinary with this image?')}).pop(0)
    return [cap1, cap2, cap3]

def auto__create_caption(IMG_SRC, CAPTION_SRC):
    if os.path.isfile(CAPTION_SRC):
        return 1
    caption_array = auto__get_AI_caption(IMG_SRC)
    write_the_file(CAPTION_SRC, json.dumps(caption_array))
    return json.dumps(caption_array)

#caption = auto__get_AI_caption('/content/drive/MyDrive/datasets/PinupFiles/Beth-Lily/beth-lily-vol-1-set-1-1.jpg')
#write_the_file('/content/test.txt', caption)
#print(caption)
#image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [ ]:
#@title Optional: Unzipping all ZIP files in IMAGE_FOLDERS
#@markdown Unzip all folders in IMAGE_FOLDERS
from zipfile import ZipFile
import os

for item in os.listdir(IMAGE_FOLDERS):
    zips = os.path.join(IMAGE_FOLDERS, item)
    print(f'Extracting: {item}')
    with ZipFile(zips, 'r') as zObject:
        zObject.extractall(path=IMAGE_FOLDERS)

In [ ]:
#@title List directories and image count
import os
for item_name in os.listdir(IMAGE_FOLDERS):
    item = os.path.join(IMAGE_FOLDERS, item_name)
    if os.path.isdir(item):
        if "_captions" in item_name:
            continue;
        print( f'{item_name}: {ret_directoryFileCount(item)}' )

In [12]:
#@title CREATE LOOP - Captioning loop for all folders within IMAGE_FOLDERS
import os; import random; import json; import time; from IPython.display import clear_output;

LOGFILE_NAME = '/' + time.strftime("%Y%m%d-%H%M%S") + '.log'

iteration = 0;
for IMAGE_FOLDER in os.listdir(IMAGE_FOLDERS):
    FOLDER = os.path.join(IMAGE_FOLDERS, IMAGE_FOLDER)
    if os.path.isdir(FOLDER):
        if "_captions" in FOLDER:
            continue;
        os.makedirs(FOLDER + '_captions', exist_ok=True)
        for FILE_NAME in os.listdir(FOLDER):
            IMG_SRC = os.path.join(FOLDER, FILE_NAME)
            if os.path.isfile(IMG_SRC):
                f_name, f_ext = os.path.splitext(FILE_NAME)
                if f_ext == '.txt':
                    write__logfile(IMAGE_FOLDERS + LOGFILE_NAME, f'{f_name}: not an image file, skipping')
                    continue;
                CAPTION_SRC = FOLDER + f'_captions/{f_name}.txt'
                caption = auto__create_caption(IMG_SRC, CAPTION_SRC)
                if caption == 1:
                    write__logfile(IMAGE_FOLDERS + LOGFILE_NAME, f'{f_name}: {caption}')
                else:
                    print(f'{f_name}: {caption}')
    iteration += 1
    if iteration == 100:
        iteration = 0
        clear_output()

#    if IMAGE_FOLDER == 'Holly-Peers':
#        print('Found Holly-Peers')
#        break

In [56]:
#@title ZIP Captioning folders, move them into _captions folder
import os
%cd {IMAGE_FOLDERS}

cap_folders = []
for item_name in os.listdir(IMAGE_FOLDERS):
    item = os.path.join(IMAGE_FOLDERS, item_name)
    if "_captions" in item_name:
        if os.path.isdir(item):
            cap_folders.append(item_name)
            continue;

for cap_folder_name in cap_folders:
    cap_folder_name_zip = os.path.join(IMAGE_FOLDERS, cap_folder_name) + '.zip'
    cap_folder          = os.path.join(IMAGE_FOLDERS, cap_folder_name)
    zipCount = create_zip_from_directory(cap_folder_name_zip, cap_folder)
    print( f'Zip created, {zipCount} files in {cap_folder_name_zip}' )

print('All caption folders are zipped')

import shutil
destDir = IMAGE_FOLDERS + '/_captions';
os.makedirs(destDir, exist_ok=True)
for cap_folder_name in cap_folders:
    zipFile = os.path.join(IMAGE_FOLDERS, cap_folder_name + ".zip")
    destFile = os.path.join(destDir, cap_folder_name + ".zip")
    shutil.move(zipFile, destFile)

print('All caption zips are located in ./_captions')

/content/drive/MyDrive/datasets/metart
Zip created, 130 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-15_PRESENTING-ANNA-SUGAR-ANNA-SUGAR-by-CECELIA_high_captions.zip
Zip created, 115 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-02_VELVET-TOUCH-ALICE-KELLY-by-NUDERO_high_captions.zip
Zip created, 121 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-18_BEAUTY-REGIME-NINA-SPHINX-by-TORA-NESS_high_captions.zip
Zip created, 121 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-30_IN-STYLE-KIERE-by-TORA-NESS_high_captions.zip
Zip created, 91 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-30_SAUCY-SMILE-POLLY-WHITE-by-CECELIA_high_captions.zip
Zip created, 122 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-11-30_SHORE-STROLL-LENI-DOLL-by-LUCA-HELIOS_high_captions.zip
Zip created, 102 files in /content/drive/MyDrive/datasets/metart/MetArt_2022-08-17_HALTER-MILA-AZUL-by-ALEX-LYNN_0a2fb_high_captions.zip
